시리즈로 받아서 로우별로 처리하고 시리즈로 반환

In [2]:
import numpy as np
import pandas as pd
import os, json
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
from konlpy.tag import Okt

In [3]:
santafe = 'extracted/santafe/'
file_seek = '/raw/dcinside/'

In [12]:
def collecting_basic_metadata(car_folder_path:str):
    file_seek = '/raw/dcinside/'
    car_basic_metadata = {}
    for folder in tqdm(os.listdir(car_folder_path)):
        if folder.startswith('._'): continue
        elif folder == '.DS_Store': continue
        date = folder
        # print(folder)
        car_basic_metadata[date] = []
        for inner_folder in os.listdir(car_folder_path + folder + file_seek):
            if inner_folder.startswith('._'): continue
            # print(inner_folder)
            post_data = {
            'title': [],
            'content': [],
            'views': [],
            'vote_sum': [],
            'num_comments': []
            }
            with open(car_folder_path + date + file_seek + inner_folder) as f:
                metadata = json.load(f)
                post_data['title'] = metadata['title']
                post_data['content'] = metadata['content']
                post_data['views'] = metadata['view_count']
                post_data['vote_sum'] = metadata['upvote_count'] - metadata['downvote_count']
                post_data['num_comments'] = metadata['comment_count']
            car_basic_metadata[date].append(post_data)
    return car_basic_metadata

In [13]:
santafe_data = collecting_basic_metadata(santafe)

100%|██████████| 140/140 [00:00<00:00, 1777.51it/s]


In [14]:
santafe_data

{'23-08-09': [{'title': '싼타페 좀 손본 이미지보니까 진심 현대 디자이너들은 ㅋㅋ',
   'content': '아래처럼 나왔으면 판매량 200프로될듯ㅋㅋ',
   'views': 156,
   'vote_sum': 0,
   'num_comments': 4},
  {'title': '싼타페 풀체의 테라스.jpg',
   'content': '후방을 최대한 각지게\n= 1bay\xa0 forest house 유사\n게이트 오픈시\xa0 자연과의 일체감을 추구',
   'views': 227,
   'vote_sum': 0,
   'num_comments': 2},
  {'title': '사막위 싼타페',
   'content': '아아',
   'views': 155,
   'vote_sum': -2,
   'num_comments': 2},
  {'title': '싼타페 쏘렌토 이런건 넘크다',
   'content': '투싼 스포티지 사이즈가 딱 적당함',
   'views': 184,
   'vote_sum': -1,
   'num_comments': 0},
  {'title': '싼타페 탐켄치 오너 어제 토레스 타보고 느낀점',
   'content': '구형 싼하 할인할때 사서 풀옵 사천 가까이 줌\n딱 토레스 나올 시점에 싼하 안팔려서 할인해줄때\n샀는데 어제 친구 토레스 높은등급? 깡통 타보고\n처참한 심경을 느껴서 글써봄\n1. 주행성능\n이건 당연히 내차가 나음. 근데 토레스가 재원 다\n낮은데 초반 가속치는 느낌은 더 나음. 후반 가속은\n당연히 비교불가. 여기서부터 뭔가 진거같은 기분임\n2. 정숙성 및 승차감\n닥치고 토레스 승. ㅅㅂ 하브 조용하다매 ㄱㅅㅂ\n3. 연비\n개시바 토레스로 바다가는데 덜 막혀서 그런가\n연비 17나옴 개 ㅆㅂ 왜 하브랑 똑같은 연비냐고\n근데 이건 잘나온 경우라고 함. 평균 14래\n근데 내 하브도 연비 안나올땐 10도 나온다\n생각하니 ㅆㅂ? 친구 멱살잡을뻔함\n4. 생긴거\n닥치고 토

In [37]:
def data_to_series(data:dict, key:str):
    """
    key -> 'title' | 'content' | 'both'
    """
    text_list = []
    sorted_dates = sorted(data.keys(), key=lambda x: datetime.datetime.strptime(x, "%y-%m-%d")) 
    for date in sorted_dates:
        if key == 'both':
            for post in data[date]:
                text_list.append(post['title'])
                text_list.append(post['content'])
        else:
            for post in data[date]:
                text_list.append(str(post[key]))
    return pd.Series(text_list)
                

In [38]:
series_data = data_to_series(santafe_data, 'title')

In [39]:
series_data

0                  싼타페 기본형 vs 투싼 풀옵 뭐사지
1                   싼타페랑 동급인 기아차 없지 않냐?
2         싼타페 후미등 led로 가자고 400써서 트림올리는거
3                싼타페 쏘렌토 가격 얼마나 오를거 같냐?
4                              싼타페dm 어떰
                     ...               
2461            개념글 싼타페 실드치는 현빠는 돌아버린거냐
2462          싼타페 솔직히 존나 이쁜거같다 쏘렌토 목 딸듯
2463     싼타페dm팔고 15년식 모하비 중고로 사는거 어떠씁니까
2464                     싼타페 옵션 뭐 같네...
2465    신형 싼타페 하브 깡통 vs 스포티지 하브 풀옵 뭐사냐?
Length: 2466, dtype: object

In [52]:
okt=Okt()
okt.nouns("차값 돈낭비 싼타페 싼타페 막상 출고하고 실물 보면 예쁠 듯 가격쌈 안비쌈 싸다 비싸다 비쌈 싸네?")

['차값', '돈', '낭비', '싼타페', '싼타페', '막상', '출고', '실물', '듯', '가격', '쌈']

In [43]:
def tokenize_korean(series_data:pd.Series):
    okt = Okt()
    tokenized_series = []
    for sentence in series_data.items(): # 명사만 추출, (문장 단위 고려 ㄴㄴ)
        try:
            tokenized_series.append(okt.nouns(str(sentence)))
        except:
            print("ERROR:", sentence)
    return pd.Series(tokenized_series)

In [44]:
tokenized_data = tokenize_korean(series_data)

In [45]:
tokenized_data

0                 [싼타페, 기본, 투싼, 풀옵, 뭐, 사지]
1                            [싼타페, 급, 기아차]
2                      [싼타페, 후미, 등, 로, 트림]
3                      [싼타페, 쏘렌토, 가격, 얼마나]
4                                    [싼타페]
                       ...                
2461                   [개념, 글, 싼타페, 실드, 현]
2462                     [싼타페, 존나, 쏘렌토, 목]
2463                    [싼타페, 팔고, 모하비, 중고]
2464                          [싼타페, 옵션, 뭐]
2465    [신형, 싼타페, 하브, 깡통, 스포티지, 하브, 풀옵, 뭐]
Length: 2466, dtype: object

In [ ]:
word_category = {
    'design' : ['외관', '외형', '내장재', '생김새', '신형', '내부', '사이즈', 
                '트렁크', '페이스리프트', '페리', '풀체인지', '풀체'
                '백라이트', '시트', '후미등', '인테리어', '실내', '실물'],
    
    'spec' : ['성능', '연비', '속도', '승차감', '마력', '엔진', '파워트레인', 'dct',
              '네비', '출력', '블박', '크루즈', 'hud', '내구성', '배터리'],
    
    'price' : ['가격', '고가', '저가', '저렴', '가성비', '트림'
               '창렬', '혜자', '옵션', '유지비', '구매'
               '가성비', '낭비', '가격', '차값', '풀옵', '할인', '깡통', '그돈씨'],
    
    'fault' : ['결함', '고장', '급발진', '하자', 'iccu', 'ecu', '누유']
}

In [53]:
okt.nouns('급발진 iccu')

['급발진']

In [ ]:
category_cnt = {
    'design' : 0,
    'spec' : 0,
    'price' : 0,
    'fault' : 0
}

for i, nouns in tqdm(tokenized_data.items()):
    # print(type(nouns))
    for n in list(nouns):
        # print(n, end=' ')
        if n in word_category['design']: category_cnt['design'] += 1
        elif n in word_category['spec']: category_cnt['spec'] += 1
        elif n in word_category['price']: category_cnt['price'] += 1
        elif n in word_category['fault']: category_cnt['fault'] += 1

2466it [00:00, 59887.75it/s]

<class 'list'>
싼타페 기본 투싼 풀옵 뭐 사지 <class 'list'>
싼타페 급 기아차 <class 'list'>
싼타페 후미 등 로 트림 <class 'list'>
싼타페 쏘렌토 가격 얼마나 <class 'list'>
싼타페 <class 'list'>
중고 싼타페 좀 <class 'list'>
싼타페 차이 임 <class 'list'>
싼타페 풀 체인지 예약 일 언제 공개 함 <class 'list'>
싼타페 먼저 <class 'list'>
울산 싼타페 위장 막 자주 <class 'list'>
싼타페 쏘랜토 신형 곧 별 관심 <class 'list'>
싼타페 신형 판매량 그랜저 듯 <class 'list'>
싼타페 휠 베이스 <class 'list'>
내 토레스 새끼 차 디딸 싼타페 <class 'list'>
지금 싼타페 하이브리드 얘 <class 'list'>
싼타페 중고차 <class 'list'>
싼타페 엔진 라인업 <class 'list'>
싼타페 끝물 가성 비좆 <class 'list'>
싼타페 추천 좀 <class 'list'>
싼타페 사서 존나 <class 'list'>
싼타페 취향 저격 <class 'list'>
싼타페 디엠 <class 'list'>
싼타페 후기 만하 <class 'list'>
스포티지 싼타페 급 차이 <class 'list'>
싼타페 살라 팰리사야 <class 'list'>
사이즈 싼타페 임 <class 'list'>
싼타페 개 좆 생겻 케 보임 <class 'list'>
싼타페 풀체 깡통 값 살 수 하이브리드 <class 'list'>
차 싼타페 친구 라브 라브 휠 낫다 <class 'list'>
방금 싼타페 위장 막 <class 'list'>
용산 방금 싼타페 위장 막 봄 <class 'list'>
싼타페 스포티지 신형 뭐 더 실내 <class 'list'>
투싼 스포티지 싼타페 쏘렌토 왜케 체감 <class 'list'>
볼보 싼타페 이유 <class 'list'>
중고 싼타페 더 프라임 둘 뭐삼 <cl

In [64]:
category_cnt

{'design': 511, 'spec': 31, 'price': 263, 'fault': 36}